每天有18项观测值, 每天的每项观测值有24项. 总共$12\times20=240$天

取每个月最后两天的数据作为测试集合? !还需修改

现在取连续的9小时的18项观测值的三次多项式为一个$\vec x_i$, 其维度再加上一bias为$(9\times18)*3+1=487$. 对应的系数向量$\vec w_i$维度也为487.

一共有240*24-9=5751笔数据, 姑且先写三次多项式拟合吧.

第10个小时的pm2.5值$y_i=\sum_{i=1}^{487}\vec x_i\cdot\vec w$.

Loss function 为$L(\vec w)=\sum_{i=1}^{487}(y_i - \vec w\cdot\vec x_i)^2+\lambda\|\vec w\|^2$

In [80]:
import re
import numpy as np
import math

train_data_path = '/Users/kyoma/Documents/meachineLearning/data/regression_pm2.5/train.csv'
f = open(train_data_path, 'r', encoding='big5')
data = []
for i in range(0, 18):
    data.append([])
i = 0
for line in f.readlines():
    j = 0
    for word in re.split(',|\n', line):
        if i>0 and j>2:
            if word=='':
                continue
            if word=='NR':
                data[(i-1)%18].append(0.0)
            else:
                data[(i-1)%18].append(float(word))
        j = j+1
    i = i+1

f.close


<function TextIOWrapper.close()>

先记录一下$y$值, 因为等下要进行缩放了

In [81]:
days = 12*20
hours = 24
y_data = []
for i in range(0, days*hours-9):
    y_data.append(data[9][i+9]) # 数据集里pm2.5正好在第10项
    
y_d = np.asarray(y_data)

Scaling

进行特征缩放, 对每个维度的值计算平均数$m_i$和标准差$\sigma _i$.

$x_i^r\gets\frac{x_i^r-m_i}{\sigma_i}$

用第$r$个例子中的第$i$个输入，减掉平均数$m_i$，然后除以标准差$\sigma_i$，得到的结果是所有的mean是0，所有的方差都是1.

$\sigma = \sqrt{\frac{1}{N}\sum{(x_i-\mu}))^2}$

In [82]:
for i in range(0, len(data[0])):
    summ = 0.0
    for j in range(0, len(data)):
        summ = summ + data[j][i]
    m = summ/len(data)
    summ = 0.0
    for j in range(0, len(data)):
        summ = summ + (data[j][i]-m)**2
    sigma = math.sqrt(summ/len(data))
    for j in range(0, len(data)):
        data[j][i] = (data[j][i]-m)/sigma

x_d每行为一个$\vec x_i$

In [83]:
x_data = []
for i in range(0, days*hours-9): # 枚举起点
    x_data.append([])
    for j in range(0, 9):
        for k in range(0, 18):
            x_data[i].append(data[k][i+j])
            x_data[i].append(data[k][i+j]**2)
            x_data[i].append(data[k][i+j]**3)
    x_data[i].append(1) # for bias

x_d = np.asarray(x_data)

$\frac{\partial L}{\partial \vec w}=\sum_{i=1}^{n}-2(y_i - \vec w\cdot\vec x_i)+2\lambda\vec w$

To update:

$\vec w^{i+1} = w^i - \eta\frac{\partial L}{\partial \vec w}$

使用Adagrad算法:

$\vec w^{t+1}\gets w^t -\frac{\eta^t}{\sigma^t}g^t$

$g^t=\frac{\partial L}{\partial\vec w},\sigma^t$:之前参数的所有微分的均方根

In [139]:
w = np.random.rand(len(x_d[0]))*10

In [ ]:
eta = 0.00001
lamb = 10
sum_w = 0.0
iteration = 100000000
for i in range(iteration):
    loss = 0.0
    par_w = 0.0
    for j in range(0, len(x_d)):
        loss = loss + ( y_d[j] - np.dot(w, x_d[j]) )**2
        par_w = par_w -( y_d[j]-np.dot(w, x_d[j]) )

    loss = loss + lamb*np.dot(w, w)
    par_w = par_w + lamb*w
    
    sum_w = sum_w + np.dot(par_w, par_w)
    sigma = math.sqrt( sum_w/(i+1) )
    w = w - eta/sigma*par_w
    #w = w - eta*par_w
    if i%1000 == 0:
        print(sum_w, np.dot(par_w, par_w), loss)